In [29]:
from PyPDF2 import PdfReader
import os
import re
import pandas as pd
import numpy as np

path = "data/Key_Stats/"
files = os.listdir(path)

dir1 = path + files[178]

In [8]:
# tokenize the string with distance index
def tokenize(string):
    string = string.lower()
    # keep english and numbers with '%'

    string = re.sub(r'[^a-z0-9% ]', ' ', string)
    string = re.sub(r' +', ' ', string)
    # delete empty string
    string = string.strip()
    return string.split(' ')

def read_pages(dir):
    text = ''
    pdf = PdfReader(dir)
    for page in pdf.pages:
        text += page.extract_text()
    return text

def full_text(text):
    text = re.sub(r'[^\x00-\x7F]+',' ', text) # only english
    text = re.sub(r'[\n\t]+', ' ', text) # delete '\n' and '\t'
    text = re.sub(r' +', ' ', text) # delete multiple spaces
    return text

def file_id(file):
    pattern = r'[A-Z]{3,3}[0-9]{3,3}'
    return re.findall(pattern, file)

In [24]:
# divide the full text
section_dict = {'fact':['quick facts'], \
                'strategy':['objective and investment strategy', 'objectives and investment strategy'], \
                'derivative':['use of derivatives', 'investment in derivatives'], \
                'risk':['what are the key risks'], \
                'performance':['how has the sub-fund performed?','how has the sub -fund performed?', 'how has the fund performed'], \
                'guarantee':['is there any guarantee'], \
                'fees':['what are the fees and charges?', 'what are the fees?'], \
                'misc':['additional information']
                }

# divide the full text
'''
def division(text):
    section = {}
    for key in section_dict['fact']:
        if key in text:
            pattern = '|'.join(key)
            temp = re.split(pattern, text, flags=re.IGNORECASE)[0]
            text = re.split(pattern, text, flags=re.IGNORECASE)[1]
            section['head'] = temp
            break
    for key in section_dict['strategy']:
        if key in text:
            pattern = '|'.join(key)
            temp = re.split(pattern, text, flags=re.IGNORECASE)[0]
            text = re.split(pattern, text, flags=re.IGNORECASE)[1]
            section['fact'] = temp
            break
    for key in section_dict['derivative']:
        if key in text:
            pattern = '|'.join(key)
            temp = re.split(pattern, text, flags=re.IGNORECASE)[0]
            text = re.split(pattern, text, flags=re.IGNORECASE)[1]
            section['strategy'] = temp
            break
    
    return section
'''

def division(text):
    section = {}
    for key in section_dict['fact']:
        if key in text:
            temp = text.split(key)[0]
            text = text.split(key)[1]
            section['head'] = temp
            break
    for key in section_dict['strategy']:
        if key in text:
            temp = text.split(key)[0]
            text = text.split(key)[1]
            section['fact'] = temp
            break
    for key in section_dict['derivative']:
        if key in text:
            temp = text.split(key)[0]
            text = text.split(key)[1]
            section['strategy'] = temp
            break
    for key in section_dict['risk']:
        if key in text:
            temp = text.split(key)[0]
            text = text.split(key)[1]
            section['derivative'] = temp
            break
    for key in section_dict['performance']:
        if key in text:
            temp = text.split(key)[0]
            text = text.split(key)[1]
            section['risk'] = temp
            break
    for key in section_dict['guarantee']:
        if key in text:
            temp = text.split(key)[0]
            text = text.split(key)[1]
            section['performance'] = temp
            break
    for key in section_dict['fees']:
        if key in text:
            temp = text.split(key)[0]
            text = text.split(key)[1]
            section['guarantee'] = temp
            break
    for key in section_dict['misc']:
        if key in text:
            temp = text.split(key)[0]
            text = text.split(key)[1]
            section['fees'] = temp
            section['misc'] = text
            break
    return section

In [62]:
full2 = full1.lower()
a = division(full2)



In [30]:
# build the corpus
corpus = pd.DataFrame(columns=['id', 'head', 'fact', 'strategy', 'derivative', 'risk', 'performance', 'guarantee', 'fees', 'misc'])

for i in range(len(files)):
    doc_dir = path + files[i]
    raw = read_pages(doc_dir)
    text = full_text(raw)
    doc_id = file_id(files[i])
    section = division(text.lower())
    data = pd.DataFrame({'id':doc_id, 'head':section['head'], 'fact':section['fact'], 'strategy':section['strategy'], 'derivative':section['derivative'], 'risk':section['risk'], 'performance':section['performance'], 'guarantee':section['guarantee'], 'fees':section['fees'], 'misc':section['misc']}, index=[i])
    corpus = pd.concat([corpus, data], axis=0)
    print(i)
    corpus.to_csv('data/corpus.csv', index=False)

0
1
2


XRef object at 221783 can not be read, some object may be missing


3


KeyError: 'derivative'

In [9]:
corpus = pd.DataFrame(columns=['id', 'head', 'fact', 'strategy', 'derivative', 'risk', 'performance', 'guarantee', 'fees', 'misc'])
i = 178
doc_dir = path + files[i]
raw = read_pages(doc_dir)
text = full_text(raw)
doc_id = file_id(files[i])
section = division(text.lower())
data = pd.DataFrame({'id':doc_id, 'head':section['head'], 'fact':section['fact'], 'strategy':section['strategy'], 'derivative':section['derivative'], 'risk':section['risk'], 'performance':section['performance'], 'guarantee':section['guarantee'], 'fees':section['fees'], 'misc':section['misc']}, index=[i])
corpus = pd.concat([corpus, data], axis=0)
print(i)
corpus.to_csv('data/corpus.csv', index=False)

178


In [22]:
text.find('performed?')

27090

In [27]:
files[i]

'3.5+Product_Key_Facts_China+Opportunities+Flexifund_EN_202.pdf'